In [2]:
from bs4 import BeautifulSoup
import configparser
import requests
import pandas as pd
import pickle
from constants_funcs import get_session, int_or_float_or_str
import json
import numpy as np
import time
with open('constants.json', 'r') as d:
    constants = json.load(d)
sess = get_session()
sess


creating new login session...


In [485]:
def build_base_player(id) -> dict:
    # try:
    url = constants['LLHEADER'] + f'/profiles.php?{str(id)}'
    main_data = sess.get(url)
    return build_player_from_soup(main_data)
    # except:
    #     print(f'error on {id}')
    #     return None
def build_player_from_soup(data):
    id = int_or_float_or_str('?'.join(data.url.split('?')[1:]))
    soup = BeautifulSoup(data.text, 'html.parser')
    ret = {'id': id, 'deceased': False}
    a, b = 'This player is inactive.' in soup.text, 'This is not an active player account.' in soup.text
    if a or b:
        ret['current_status'] = 'deactivated'
        ret['username'] = ret['id'] if b else soup.find('div', attrs={'class': 'namediv'}).text.strip()
    else:
        box = soup.find('div', attrs={'class': 'topcont'})
        if '\t\tPassed away' in box.text:
            ret['current_status'] = 'deactivated'
            ret['deceased'] = True
        else:
            ret['current_status'] = box['class'][-1]
        branch_comp = soup.findAll('div', attrs={'class': 'demog_row'})[1]
        ret['branch_id'] = int(branch_comp.find('a')['href'].split('?')[-1])
        ret['branch_name'] = branch_comp.text.strip().split(': ')[1]
        genloc = soup.findAll('div', attrs={'class': 'demog_row'})[0].text.strip().split('\n\n')
        ret['college'] = (soup.find(text='College:').parent.parent.contents[2].strip()
            if soup.find(text='College:') else None)
        ret['gender'] = (soup.find(text='Gender:').parent.parent.text.split(':')[1].strip()
            if soup.find(text='Gender:') else None)
        ret['location'] = (soup.find(text='Location:').parent.parent.text.split(':')[1].strip()
            if soup.find(text='Location:') else None)
        ret['referrer'] = (int(soup.find(text='Referrer:').parent.parent.find('a')['href'].split('?')[-1])
            if soup.find(text='Referrer:') else 0)
        ret['name'] = soup.find('h1', attrs={'class': 'namecss'}).text
        ret['league'] = soup.find('div', attrs={'class': 'leaguelogodiv'}).text.strip()
    return ret
def build_inverse():
    bu = dict()
    for key in player_profiles.keys():
        if 'name' in player_profiles[key].keys: bu[key]['name'] = bu[key]
        else: bu[key]['username'] = bu[key]
def add_id_to_dicts(new_id: str):
    new_p = build_base_player(new_id.replace('\' ', '_').replace(' ', '_'))
    with open('player_profiles.json', 'r') as fp:
        player_profiles = json.load(fp)
    with open('pp_by_username.json', 'r') as fp:
        by_username = json.load(fp)
    player_profiles[str(new_p['id'])] = new_p
    # print(new_p)
    by_username[new_p['name'].lower() if 'name' in new_p.keys() else new_p['username'].lower()] = str(new_p['id'])
    with open('player_profiles.json', 'w') as fp:
        json.dump(player_profiles, fp)
    with open('pp_by_username.json', 'w') as fp:
        json.dump(by_username, fp)
    return new_p['id']

In [486]:
with open('pp_by_username.json', 'r') as fp:
    by_username = json.load(fp)
with open('player_profiles.json', 'r') as fp:
    player_profiles = json.load(fp)
undo_username_chg = {
    'obrien': 'o\'brien',
    'orourkek': 'o\'rourkek',
    'olearyn': 'o\'learyn',
    'o_brienm': 'o\'brienm',
    'o_brienp': 'o\'brienp',
    'o_neillj': 'o\'neillj',
    'vaidman_stonem': 'vaidman stonem',
    'zeidlere_bang_a_gong': 'zeidlere bang_a_gong',
    'johnsonnickoclock': 'johnsonnicko\'clock',
    'goedels_incompleteness': 'goedel\'s incompleteness',
    'frankly_my_dear_i_dont': 'frankly my dear i don\'t',
    'int_veldt': 'in\'t veldt',
    'rudenskyyes_its_me': 'rudenskyyes it\'s me',
    'frisciap_child_please': 'frisciap child_please',
    'trivedid_ancient_history': 'trivedid_ancient history'
}
def username_to_id(row):
    global by_username
    u = str(row['player']).strip()
    if u.isnumeric(): return u
    elif u in undo_username_chg.keys(): u = undo_username_chg[u]
    if u not in by_username.keys() and '_' in u: u = u.replace('_', ' ')
    if u not in by_username.keys() and u[0] in ['o', 'd']: u = f'{u[0]}\'{u[1:]}'
    if u not in by_username.keys():
        id = add_id_to_dicts(u)
        with open('pp_by_username.json', 'r') as fp:
            by_username = json.load(fp)
    #     return str(id)
    return by_username[u]

def add_id_col(season):
    with open(f'./FULL_DATA/LL{season}/results.json') as fp:
        cur = json.load(fp)
    matches_history, match_participation, player_answer_history = [], [], []
    for md_rundle in cur:
        matches_history.extend(md_rundle['matches_history'])
        match_participation.extend(md_rundle['match_participation'])
        player_answer_history.extend(md_rundle['player_answer_history'])
    matches_history = pd.DataFrame(matches_history,
        columns=['match_id', 'rundle', 'league', 'div', 'season', 'md'])
    match_participation = pd.DataFrame(match_participation,
        columns=['match_id', 'season', 'md', 'rundle', 'league', 'div', 'player', 'result'])
    match_participation['player'] = match_participation['player'].str.lower()
    player_answer_history = pd.DataFrame(player_answer_history,
        columns=['match_id', 'rundle', 'league', 'div', 'season', 'md', 'qnum', 'player', 'correct', 'def_given'])
    player_answer_history['player'] = player_answer_history['player'].str.lower()
    match_participation['player_id'] = match_participation.apply(username_to_id, axis=1)
    player_answer_history['player_id'] = player_answer_history.apply(username_to_id, axis=1)
    matches_history.to_csv(f'./FULL_DATA/LL{season}/matches_history.csv')
    match_participation.to_csv(f'./FULL_DATA/LL{season}/match_participation.csv')
    player_answer_history.to_csv(f'./FULL_DATA/LL{season}/player_answer_history.csv')
    print(f'complete resaving of LL{season}')
# for s in range(30, 93): add_id_col(s)

complete resaving of LL30
complete resaving of LL31
complete resaving of LL32
complete resaving of LL33
complete resaving of LL34
complete resaving of LL35
complete resaving of LL36
complete resaving of LL37
complete resaving of LL38
complete resaving of LL39
complete resaving of LL40
complete resaving of LL41
complete resaving of LL42
complete resaving of LL43
complete resaving of LL44
complete resaving of LL45
complete resaving of LL46
complete resaving of LL47
complete resaving of LL48
complete resaving of LL49
complete resaving of LL50
complete resaving of LL51
complete resaving of LL52
complete resaving of LL53
complete resaving of LL54
complete resaving of LL55
complete resaving of LL56
complete resaving of LL57
complete resaving of LL58
complete resaving of LL59
complete resaving of LL60
complete resaving of LL61
complete resaving of LL62
complete resaving of LL63
complete resaving of LL64
complete resaving of LL65
complete resaving of LL66
complete resaving of LL67
complete res

In [47]:
player_dict_replacement = {
    'omahoney-sch.': 'omahoney-schwartzd',
    'goldsteinb_': 'goldsteinben',
    'behmkej_girlg.': 'behmkej_girlgeologist',
    'bingham-pankr.': 'bingham-pankratzd',
    'haynes-peters.': 'haynes-peterson_r',
    'mckinnon-crow.': 'mckinnon-crowleys',
    'tarantaa_aka_.': 'tarantaa_aka_abbkae',
    'frisardirule.': 'frisardirules.',
    'cordrayr': 'adamsr',
    'stoneecoldste.': 'stoneecoldsteveethan',
    'griffinv': 'griffin_chasev',
    'fergusonl27': 'fergusonl',
    'o\'learyn': 'olearyn',
    'o\'connort': 'oconnort',
    'o\'brianp': 'obrianp',
    'la porten': 'la_porten',
    'o\'rourkek2': 'orourkek2',
    'aasmuleatable.': 'aasmuleatableaasmul'
}
def check_season(season):
    with open(f'./FULL_DATA/LL{season}/results.json') as fp:
        cur = json.load(fp)
    matches_history, match_participation, player_answer_history = [], [], []
    for md_rundle in cur:
        matches_history.extend(md_rundle['matches_history'])
        match_participation.extend(md_rundle['match_participation'])
        player_answer_history.extend(md_rundle['player_answer_history'])
    matches_history = pd.DataFrame(matches_history,
        columns=['match_id', 'rundle', 'league', 'div', 'season', 'md'])
    match_participation = pd.DataFrame(match_participation,
        columns=['match_id', 'season', 'md', 'rundle', 'league', 'div', 'player', 'result'])
    match_participation['player'] = match_participation['player'].str.lower()
    # match_participation.replace({'player': player_dict_replacement}, inplace=True)
    player_answer_history = pd.DataFrame(player_answer_history,
        columns=['match_id', 'rundle', 'league', 'div', 'season', 'md', 'qnum', 'player', 'correct', 'def_given'])
    player_answer_history['player'] = player_answer_history['player'].str.lower()
    # player_answer_history.replace({'player': player_dict_replacement}, inplace=True)
    # match_participation.drop(match_participation[(
    #     bool(match_participation.match_id))
    #     & (match_participation.player == 'la_porten')
    #     # & (match_participation.season == 52)
    #     ].index, inplace=True)
    # match_participation.loc[(match_participation['league'] == 'Skyline') & (match_participation['player'] == 'obrienm'), 'player'] = 'o_brienm'
    # player_answer_history.loc[(player_answer_history['league'] == 'Skyline') & (player_answer_history['player'] == 'obrienm'), 'player'] = 'o_brienm'
    # matches_history.drop_duplicates(inplace=True), match_participation.drop_duplicates(
    #     inplace=True), player_answer_history.drop_duplicates(inplace=True)
    mh_v = matches_history.match_id.value_counts()
    mh_dups = matches_history[matches_history.match_id.isin(mh_v.index[mh_v.ne(1)])]
    print(f'season {season} duplicates in match hist: {(mh_dups.shape[0])}')
    mp_v = match_participation.player.value_counts()
    mp_dups = match_participation[match_participation.player.isin(mp_v.index[mp_v.ne(25)])]
    print(f'season {season} duplicates in match part: {(mp_dups.shape[0])}')
    pah_v = player_answer_history.player.value_counts()
    pah_dups = player_answer_history[player_answer_history.player.isin(pah_v.index[pah_v.ne(150)])]
    print(f'season {season} duplicates in ans hist: {(pah_dups.shape[0])}')
    return mh_dups, mp_dups, pah_dups
def check_resaved_ids(season):
    match_participation = pd.read_csv(f'./FULL_DATA/LL{season}/match_participation.csv', header=0)
    player_answer_history = pd.read_csv(f'./FULL_DATA/LL{season}/player_answer_history.csv', header=0)
    mp_v = match_participation.player_id.value_counts()
    mp_dups = match_participation[match_participation.player_id.isin(
        mp_v.index[mp_v.lt(20 if season > 38 else 20)])]
    print(f'season {season} duplicates in match part: {(mp_dups.shape[0])}')
    pah_v = player_answer_history.player_id.value_counts()
    pah_dups = player_answer_history[player_answer_history.player_id.isin(
        pah_v.index[pah_v.lt(100)])]
    print(f'season {season} duplicates in ans hist: {(pah_dups.shape[0])}')
    return mp_dups, pah_dups
# for s in range(72, 83): check_resaved_ids(s)
mp_dups, pah_dups = check_resaved_ids(72)
# mh_dups, mp_dups, pah_dups = check_season(82)
# for s in range(30, 39):
#     mh_dups, mp_dups, pah_dups = check_season(s)

season 72 duplicates in match part: 0
season 72 duplicates in ans hist: 0


In [46]:
match_participation = pd.read_csv(f'./FULL_DATA/LL72/match_participation.csv', header=0)
match_participation.loc[match_participation.player_id.apply(lambda x: not str(x).isnumeric())]
match_participation.loc[
    match_participation.player_id.apply(lambda x: str(x) == 'o\'\'brienm'),
    ['player']] = 'o\'brienm'
match_participation.loc[
    match_participation.player_id.apply(lambda x: str(x) == 'o\'\'brienm'),
    ['player_id']] = '11123'
# player_answer_history.loc[
#     player_answer_history.player_id.apply(lambda x: str(x) == 'o\'\'neillj'),
#     ['player']] = 'o\'neillj'
match_participation.loc[match_participation.player_id.apply(lambda x: not str(x).isnumeric())]
# player_answer_history.loc[
#     player_answer_history.player_id.apply(lambda x: str(x) == 'o\'\'neillj'),
#     ['player_id']] = '20500'
# match_participation.loc[
#     match_participation.player_id.apply(lambda x: str(x) == 'd\'uff-obryans'),
#     ['player_id']] = '13265'
# player_answer_history.to_csv(f'./FULL_DATA/LL72/player_answer_history.csv', index=False)
match_participation.to_csv(f'./FULL_DATA/LL72/match_participation.csv', index=False)

/usr/local/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [575]:
match_participation = pd.read_csv(f'./FULL_DATA/LL71/match_participation.csv', header=0)
match_participation.loc[match_participation.player_id.apply(lambda x: not str(x).isnumeric())]
# player_answer_history = pd.read_csv(f'./FULL_DATA/LL71/player_answer_history.csv', header=0)
# player_answer_history.loc[player_answer_history.player_id.apply(lambda x: not str(x).isnumeric())]
for season in range(71, 81):
    # match_participation = pd.read_csv(f'./FULL_DATA/LL{season}/match_participation.csv', header=0)
    # player_answer_history = pd.read_csv(f'./FULL_DATA/LL{season}/player_answer_history.csv', header=0)
#     # player_answer_history.astype({'player_id': 'string'})
#     # player_answer_history[]
#     player_answer_history.loc[
#         player_answer_history.player_id.apply(lambda x: str(x) == 'o\'\'brienm'),
#         ['player']] = 'o\'brienm'
    # match_participation.loc[
    #     match_participation.player_id.apply(lambda x: str(x) == 'sipesd$'),
    #     ['player_id']] = '27242'
    # player_answer_history.loc[
    #     player_answer_history.player_id.apply(lambda x: str(x) == 'sipesd$'),
    #     ['player_id']] = '27242'
    # player_answer_history.to_csv(f'./FULL_DATA/LL{season}/player_answer_history.csv', index=False)


SyntaxError: unexpected EOF while parsing (<ipython-input-575-9be2621c8dd9>, line 20)

In [606]:
with open(f'./FULL_DATA/LL80/results.json') as fp:
    cur = json.load(fp)
matches_history, match_participation, player_answer_history = [], [], []
for md_rundle in cur:
    matches_history.extend(md_rundle['matches_history'])
    match_participation.extend(md_rundle['match_participation'])
    player_answer_history.extend(md_rundle['player_answer_history'])
len(matches_history), len(match_participation), len(player_answer_history)

(169450, 338900, 2033400)

In [213]:
len(list(filter(lambda m: m[-1] == 'U', match_participation)))

2

In [500]:
mp_v = mp_dups.player.value_counts()
# mp_dups.loc[(mp_dups['league'] == 'Skyline') & (mp_dups['player'] == 'obrienm'), 'player'] = 'o_brienm'
mp_dups.sort_values(['md', 'player'], ascending=True)
# set(mp_dups.player)
# mp_v.head(100)
# mp_v[mp_v.index[mp_v.ne(1)]]
# mp_dups
# mp_dups[mp_dups['player'] == 'sheridana'].sort_values('md')
# mp_dups.loc[(mp_dups['league'] == 'Horizon') & (mp_dups['player'] == 'batesc')].sort_values(['md', 'player'], ascending=True)

,Unnamed: 0,match_id,season,md,rundle,league,div,player,result,player_id
236071,236071,2829799,82,1,R,Coastal,0,34162,L,34162
141860,141860,2823375,82,1,A,Juniper,0,oconnors,W,34162
49379,49379,2837347,82,2,R,Coastal,0,34162,W,34162
238288,238288,2830925,82,2,A,Juniper,0,oconnors,L,34162
192473,192473,2844895,82,3,R,Coastal,0,34162,W,34162
51548,51548,2838475,82,3,A,Juniper,0,oconnors,W,34162
5968,5968,2852445,82,4,R,Coastal,0,34162,T,34162
242144,242144,2846025,82,4,A,Juniper,0,oconnors,W,34162
290742,290742,2859995,82,5,R,Coastal,0,34162,L,34162
8170,8170,2853575,82,5,A,Juniper,0,oconnors,W,34162


In [52]:
merge_thing = 'player_answer_history'
d1 = pd.read_csv(f'./FULL_DATA/LL30/{merge_thing}.csv', header=0)
for season in range(31, 94):
    d2 = pd.read_csv(f'./FULL_DATA/LL{season}/{merge_thing}.csv', header=0)
    d1 = pd.concat([d1, d2], axis=0)
    if season % 10 == 0: print(f'concated season {season}')
d1.drop('Unnamed: 0', axis=1, inplace=True)
d1.head(), d1.shape

concated season 40
concated season 50
concated season 60
concated season 70
concated season 80
concated season 90


(     match_id rundle  league  div  season  md  qnum player correct  def_given  \
 0  25904701.0      A  League    0      30   1     1  white    True        2.0   
 1  25904701.0      A  League    0      30   1     2  white   False        3.0   
 2  25904701.0      A  League    0      30   1     3  white    True        1.0   
 3  25904701.0      A  League    0      30   1     4  white   False        1.0   
 4  25904701.0      A  League    0      30   1     5  white   False        2.0   
 
    player_id  
 0      19962  
 1      19962  
 2      19962  
 3      19962  
 4      19962  ,
 (65442036, 11))

In [53]:
d1.to_csv(f'./FULL_DATA/{merge_thing}.csv', index=False)

In [54]:
mh_v = d1[['md', 'season', 'qnum', 'player_id']].value_counts()
mh_dups = d1[d1.match_id.isin(mh_v.index[mh_v.ne(1)])]
mh_dups
# set(d1['season'])

,match_id,rundle,league,div,season,md,qnum,player,correct,def_given,player_id
